# Data Engineer Certification Sample Practical Exam

HappyPaws, creates fun and educational apps for pet owners. 

HappyPaws wants to help pet owners understand their pets better by tracking their activities and health through the app. 

The data engineering team is responsible for making sure all the pet data from thousands of users is organized and safe, so pet owners can get tips to keep their pets happy and healthy.


## Task

HappyPaws has collected three datasets over the past year: 
 - "pet_activities.csv" which logs daily activities of pets, 
 - "pet_health.csv" which records vet visits and health issues, and 
 - "users.csv" which contains information about the pet owners. 

Each dataset contains unique identifiers for pets and/or their owners. 

The engineers developing the app currently write code to cross reference all of these data sources. 

They want to make things easier by having a single table with all data included.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset. 

The final dataset should provide a comprehensive view of each pet's activities, health records, and owner information. 

- To test your code, your manager will run only the code `all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')`
- Your `all_pet_data()` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.

## Data

The data that has been provided has the following structure

The function that you write must return data as described below. There should be a unique row for each activity/health visit. 

Where missing values are permitted, they should be in the default Python format.

|Column Name | Description |
|-------|-------|
| pet_id | Unique identifier for each pet. There should not be any missing values.|
| date | The date of the activity recorded or the date of the health visit, in date format. There should not be any missing values.|
| activity_type | The type of activity, one of 'Walking', 'Playing', 'Resting' or for rows that relate to a health visit, the value 'Health'. Missing values are permitted.|
| duration_minutes | The duration of the activity in minutes. For rows that relate to health visits, this should be 0. Missing values for other activities are permitted.|
| issue | The health issue identified or check-up note. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| resolution | The outcome or advice given for the issue. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| owner_id | Unique identifier for the pet owner. All pets must have an owner.|
| owner_age_group | The age group of the owner (e.g., 18-25, 26-35, etc.). Missing values are permitted.|
| pet_type | The type of pet (e.g., Dog, Cat). Missing values are permitted.|

#### Código completo

In [ ]:
import pandas as pd
import numpy as np

def all_pet_data(activities_file, health_file, users_file):
    activities = pd.read_csv(activities_file)
    health = pd.read_csv(health_file)
    users = pd.read_csv(users_file)

    # alguns registros usam a versão curta do nome, precisa padronizar
    activity_map = {
        'Walking': 'Walking', 'Walk': 'Walking',
        'Playing': 'Playing', 'Play': 'Playing',
        'Resting': 'Resting', 'Rest': 'Resting',
    }
    activities['activity_type'] = activities['activity_type'].astype(str).str.strip().map(activity_map)
    activities['date'] = pd.to_datetime(activities['date'], errors='coerce')

    # duração vem como '-' quando não foi registrada
    activities['duration_minutes'] = pd.to_numeric(
        activities['duration_minutes'].astype(str).str.replace('-', '', regex=False).str.strip(),
        errors='coerce'
    )

    activities['issue'] = np.nan
    activities['resolution'] = np.nan
    df_activities = activities[['pet_id', 'date', 'activity_type', 'duration_minutes', 'issue', 'resolution']].copy()

    health = health.rename(columns={'visit_date': 'date'})
    health['date'] = pd.to_datetime(health['date'], errors='coerce')
    health['activity_type'] = 'Health'
    health['duration_minutes'] = 0

    for col in ['issue', 'resolution']:
        health[col] = health[col].astype(str).str.strip()
        health[col] = health[col].replace({'Nan': np.nan, 'None': np.nan, 'nan': np.nan, '': np.nan})

    df_health = health[['pet_id', 'date', 'activity_type', 'duration_minutes', 'issue', 'resolution']].copy()

    df = pd.concat([df_activities, df_health], ignore_index=True)
    df = df.dropna(subset=['pet_id', 'date'])
    df['pet_id'] = df['pet_id'].astype(int)

    users['owner_age_group'] = users['owner_age_group'].astype(str).str.strip().replace(
        {'Nan': np.nan, 'None': np.nan, 'nan': np.nan, '': np.nan}
    )
    users['pet_type'] = users['pet_type'].astype(str).str.strip().str.title().replace(
        {'Nan': np.nan, 'None': np.nan, 'nan': np.nan, '': np.nan}
    )

    users = users[['owner_id', 'pet_id', 'owner_age_group', 'pet_type']].copy()
    users['owner_id'] = users['owner_id'].astype(int)
    users['pet_id'] = users['pet_id'].astype(int)

    df = df.merge(users, on='pet_id', how='left')
    df = df.dropna(subset=['owner_id'])
    df['owner_id'] = df['owner_id'].astype(int)

    df = df[['pet_id', 'date', 'activity_type', 'duration_minutes',
              'issue', 'resolution', 'owner_id', 'owner_age_group', 'pet_type']]

    df['duration_minutes'] = pd.to_numeric(df['duration_minutes'], errors='coerce')

    return df